<a href="https://colab.research.google.com/github/wesslen/data-management/blob/main/notebooks/02_gemini_prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/wesslen/data-management.git

Cloning into 'data-management'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 39 (delta 13), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (39/39), 53.71 KiB | 1.45 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [2]:
import json
import os
from google.colab import drive  # If using Google Drive

# --- User sets input folder path ---
# Option 1: Local files (if you've mounted your local filesystem)
input_folder = "data-management/data/output"

# Option 2: Google Drive
# input_folder = "/content/drive/MyDrive/your_folder_name" # Replace with your actual path
# drive.mount('/content/drive')  # Mount Google Drive (if using Drive)

# --- Function to read markdown files ---
def read_markdown_file(filepath):
    try:
        with open(filepath, "r") as f:
            return f.read()
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None

# --- Read files from the specified input folder ---
def read_input_files(folder_path):
    balance_sheet_sop = read_markdown_file(os.path.join(folder_path, "balance-sheet-sop.md"))
    balance_sheet = read_markdown_file(os.path.join(folder_path, "balance-sheet.md"))
    data_management_template = read_markdown_file(os.path.join(folder_path, "data-management-template.md"))
    metadata_dictionary = read_markdown_file(os.path.join(folder_path, "metadata-dictionary.md"))

    # Check if all files were read successfully
    if None in [balance_sheet_sop, balance_sheet, data_management_template, metadata_dictionary]:
        print("Error: Could not read all required files. Check your input folder path and file names.")
        return None, None, None, None  # Return None for all if any are missing

    return balance_sheet_sop, balance_sheet, data_management_template, metadata_dictionary


balance_sheet_sop, balance_sheet, data_management_template, metadata_dictionary = read_input_files(input_folder)


def generate_prompt_with_instructions(sop, balance_sheet, template, dictionary):

    prompt = f"""
    You are a data management expert.  Populate the provided data management template with information extracted from the balance sheet and its standard operating procedures.  The metadata dictionary provides the required fields and format.

    Balance Sheet SOP:
    ```markdown
    {sop}
    ```

    Balance Sheet:
    ```markdown
    {balance_sheet}
    ```

    Data Management Template:
    ```markdown
    {template}
    ```

    Metadata Dictionary:
    ```markdown
    {dictionary}
    ```

    Output the populated data management template in JSON format adhering to the structure and formats defined in the metadata dictionary.
    """
    return prompt


def generate_prompt_with_constraints(sop, balance_sheet, template, dictionary):
    # Define the expected JSON structure based on metadata_dictionary
    # This requires parsing  metadata_dictionary.md and creating a corresponding JSON schema or example.
    # This is a placeholder and needs to be customized based on your specific dictionary format.
    # Example of inferring structure from `metadata-dictionary.md` (replace with your actual logic):
    json_structure_example = {}
    for line in dictionary.splitlines():
        if "|" in line:  # Assuming markdown table format
            parts = [p.strip() for p in line.split("|") if p.strip()]
            if len(parts) >= 2:  # Field Name | Format
                json_structure_example[parts[0]] = {"format": parts[1], "value": ""} # Placeholder for values

    prompt = f"""
    You are a data management expert.  Populate the provided data management template with information extracted from the balance sheet and its standard operating procedures. The output must strictly adhere to the following JSON structure:

    ```json
    {json.dumps(json_structure_example, indent=4)}
    ```

    Balance Sheet SOP:
    ```markdown
    {sop}
    ```

    Balance Sheet:
    ```markdown
    {balance_sheet}
    ```

    Data Management Template:
    ```markdown
    {template}
    ```

    Output the populated data management template in JSON format.
    """
    return prompt

# Example usage:

prompt_with_instructions = generate_prompt_with_instructions(
    balance_sheet_sop, balance_sheet, data_management_template, metadata_dictionary
)

prompt_with_constraints = generate_prompt_with_constraints(
    balance_sheet_sop, balance_sheet, data_management_template, metadata_dictionary
)



print("Prompt with instructions:")
print(prompt_with_instructions)

print("\nPrompt with constraints:")
print(prompt_with_constraints)


# Save the prompts to files (optional)
with open("prompt_with_instructions.txt", "w") as f:
  f.write(prompt_with_instructions)

with open("prompt_with_constraints.txt", "w") as f:
  f.write(prompt_with_constraints)

Prompt with instructions:

    You are a data management expert.  Populate the provided data management template with information extracted from the balance sheet and its standard operating procedures.  The metadata dictionary provides the required fields and format.

    Balance Sheet SOP:
    ```markdown
    # Balance Sheet Standard Operating Procedures

Standard Operating Procedures: Balance Sheet Maintenance

# 1. Standard Operating Procedures: Balance Sheet Maintenance

Document ID: FIN-SOP-001
Version: 1.0
Last Updated: November 16, 2024
Department: Finance

## 1.1 Purpose

This document outlines the standard operating procedures for maintaining and validating the company’s balance sheet. These procedures ensure accuracy, consistency, and compliance with accounting standards.

## 1.2 Scope

These procedures apply to all financial analysts responsible for maintaining and reviewing the company’s balance sheet spreadsheet.

## 1.3 Responsibilities

Financial Analysts: Daily maintena

## Call Gemini

In [7]:
!uv pip install --system google-generativeai

Using Python 3.10.12 environment at /usr
Audited 1 package in 87ms


In [15]:
import json
import os
from google.colab import drive, userdata
import google.generativeai as genai

genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))

def call_gemini_api(prompt):
    """Calls the Gemini API with the given prompt."""
    try:
        model = genai.GenerativeModel(
            model_name="gemini-1.5-flash"  # Or "gemini-1.0" etc.
        )
        response = model.generate_content(prompt)

        return response.text.strip()   # Or response.completion # Use response.completion for Pro models
    except Exception as e:
        print(f"Error calling Gemini API: {e}")
        return None


# Example usage:
prompt_with_instructions = generate_prompt_with_instructions(
    balance_sheet_sop, balance_sheet, data_management_template, metadata_dictionary
)

prompt_with_constraints = generate_prompt_with_constraints(
    balance_sheet_sop, balance_sheet, data_management_template, metadata_dictionary
)


response_instructions = call_gemini_api(prompt_with_instructions)
if response_instructions:
  try:
    # Try parsing the response as JSON
    json_response_instructions = json.loads(response_instructions)
    print("JSON Response (Instructions):")
    print(json.dumps(json_response_instructions, indent=4))

  except json.JSONDecodeError as e:
    print(f"Warning: Could not parse response as JSON: {e}")
    print("Raw Response (Instructions):")
    print(response_instructions)  # Print the raw response if it's not valid JSON


response_constraints = call_gemini_api(prompt_with_constraints)
if response_constraints:
    try:
        json_response_constraints = json.loads(response_constraints)
        print("\nJSON Response (Constraints):")
        print(json.dumps(json_response_constraints, indent=4))
    except json.JSONDecodeError as e:
        print(f"Warning: Could not parse response as JSON: {e}")
        print("Raw Response (Constraints):")
        print(response_constraints)




# Save responses to files (optional)
if response_instructions:
  with open("response_instructions.json", "w") as f:
      f.write(response_instructions) # Write raw response, even if not parsed as JSON

if response_constraints:
  with open("response_constraints.json", "w") as f:
    f.write(response_constraints)

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 637.65ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 406.36ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 4001.39ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 432.37ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 408.80ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 409.28ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 382.82ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash:genera

KeyboardInterrupt: 